In [5]:
# 재정의: 이전 변수 세션이 초기화된 것으로 판단되므로 CSV 다시 로드
df_sample = pd.read_csv("loan_data_sample.csv")
df = df_sample.copy()

# 결측치 처리 (단순한 임시 전략: 결측 생애주기 대체, 소비금액 NaN은 0)
df['LIFE_STAGE'].fillna('UNKNOWN', inplace=True)
consumption_cols = [
    'GROCERY_AM', 'HOS_AM', 'CLOTH_AM', 'RESTRNT_AM',
    'AUTOMNT_AM', 'AUTOSL_AM', 'KITWR_AM', 'FABRIC_AM',
    'ACDM_AM', 'MBRSHOP_AM'
]
df[consumption_cols] = df[consumption_cols].fillna(0)

# 총 소비 금액 계산
df['TOTAL_SPEND'] = df[consumption_cols].sum(axis=1)

# AGE와 MBR_RK 처리
df['AGE'] = pd.to_numeric(df['AGE'], errors='coerce').fillna(30).astype(int)
df['MBR_RK'] = pd.to_numeric(df['MBR_RK'], errors='coerce').fillna(25).astype(int)

# initialRate 수식 적용
def calculate_initial_rate(row):
    base = 5.0
    age_factor = 0.02 * (row['AGE'] // 10)
    digital_discount = -0.3 if row['DIGT_CHNL_USE_YN'] == 'Y' else 0.0
    spend_penalty = 0.001 * (row['TOTAL_SPEND'] / 1000)
    rank_bonus = -0.05 * (row['MBR_RK'] / 10)
    return round(base + age_factor + digital_discount + spend_penalty + rank_bonus, 2)

df['initialRate'] = df.apply(calculate_initial_rate, axis=1)

# 확인
df[['AGE', 'DIGT_CHNL_USE_YN', 'TOTAL_SPEND', 'MBR_RK', 'initialRate']].head()


,AGE,DIGT_CHNL_USE_YN,TOTAL_SPEND,MBR_RK,initialRate
0,25,Y,80,25,4.62
1,55,N,0,25,4.97
2,60,N,110,25,5.00
3,60,N,130,25,5.00
4,60,N,0,25,5.00


In [11]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import joblib

# 1. 데이터 로드
df = pd.read_csv("loan_data_sample.csv")

# 2. 전처리
df['AGE'] = pd.to_numeric(df['AGE'], errors='coerce').fillna(30).astype(int)
df['SEX_CD'] = pd.to_numeric(df['SEX_CD'], errors='coerce').fillna(1).astype(int)
df['TOTAL_SPEND'] = df[['GROCERY_AM', 'HOS_AM', 'CLOTH_AM']].fillna(0).sum(axis=1)

# 3. 초기 금리 생성 (실제 모델에서는 서비스 로직에 기반해 생성해야 함)
df['initialRate'] = (
    3.0 + (df['AGE'] / 100) * 4.0 + (df['TOTAL_SPEND'] / 10000)
).clip(3.0, 7.5).round(2)

# 4. 전체 금리 범위 확인 → 데이터 기준 min/max
global_min_rate = df['initialRate'].min()
global_max_rate = df['initialRate'].max()

# 5. 타겟 정규화
df['normalizedRate'] = (
    (df['initialRate'] - global_min_rate) / (global_max_rate - global_min_rate)
).clip(0.0, 1.0)

# 6. 학습 준비
features = ['AGE', 'SEX_CD', 'TOTAL_SPEND']
X = df[features]
y = df['normalizedRate']

# 7. 파이프라인 구성 및 학습
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
])

pipeline.fit(X, y)

# 8. 모델 및 금리 범위 저장
joblib.dump({
    "model": pipeline,
    "min_rate": global_min_rate,
    "max_rate": global_max_rate
}, "normalized_rate_model_with_bounds.pkl")


['normalized_rate_model_with_bounds.pkl']